In [225]:
#!/usr/bin/env python
# coding: utf-8

# todo: 
# 1. spatial_mask.inverse_transform, sometimes you dont have the spatial_mask store in the local() to use.
# 2. clustering within sbj to see the trend, with selected large spindle-TR sample size sbj
# 3. pca, limited number of eigenvector to feed in clustering, then be used to reconstruct data.
# 4. a handle about whether save it, eg. SpindleTR_SampleSize_N11, into the folder; also when plot it, where direct to extract
# 5. motion outliers exclusion? maybe not, check how many could be excluded because of motion

import os
import re
import glob as gb
import pandas as pd
import numpy as np
import random
from nilearn.input_data import NiftiMasker

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score, pairwise_distances, calinski_harabasz_score, davies_bouldin_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.decomposition import PCA

# from sklearn.svm import SVC
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import LeaveOneGroupOut
# from sklearn.feature_selection import SelectKBest, SelectPercentile, f_classif
# from sklearn.pipeline import Pipeline

# sbjlist = [ "CoRe_001", "CoRe_023", "CoRe_079", "CoRe_107",
# "CoRe_082", "CoRe_100", "CoRe_087", "CoRe_192", "CoRe_195",
# "CoRe_235", "CoRe_267", "CoRe_237", "CoRe_296",
# "CoRe_011", "CoRe_022", "CoRe_050", "CoRe_054", "CoRe_162", "CoRe_155", "CoRe_094"] 
# N=18 "CoRe_063", "CoRe_128", "CoRe_220", "CoRe_223", "CoRe_268", # N=8 "CoRe_102",

# sbjlist = [ "CoRe_079",
# "CoRe_082", "CoRe_100", "CoRe_195",
# "CoRe_235", "CoRe_267", "CoRe_237",
# "CoRe_011", "CoRe_022", "CoRe_054", "CoRe_094"] 

sbjlist = [ "CoRe_100"] #011

special_sbjs = {'sbj_ID': ['CoRe_001'], 
        'fMRI_sess_2use': [0]
        }

ref_special = pd.DataFrame(special_sbjs, columns = ['sbj_ID', 'fMRI_sess_2use'])

daylist = ["D1"]

data_suffix = 'MDsCoNr%mniNLS.nii.gz'
dir_root = '/Volumes/JDlab_Shuo/CoRe'

# collect all the masks intended to be used
masklist = []
for dr in os.listdir(dir_root + '/Analysis_cabinet/ROIs_mask/'):
    if dr == 'mnit1_09c_3mm_mask.nii': #
        masklist.append(dr)
mask_filename = dir_root + '/Analysis_cabinet/ROIs_mask/'+ masklist[0]       
        
# make a table to save out the data inclusion info        
df_samples = pd.DataFrame(np.zeros((len(sbjlist), 4)),
                   columns=['total_spindle', 'total_TR', 'total_sleep_sess', 'specified_sleep_sess'])

df_samples['sbjlist']=sbjlist
df_samples.specified_sleep_sess+=-1 # -1 means haven't needed to specified

save_data_SZ = 1

save_fmri_mat = 1
load_fmri_mat = 0

fmri_mat_name = dir_root + '/Spindle_CAP/fmri_all_selected_N' + str(len(sbjlist)) + '.csv'
# fmri_mat_name = dir_root + '/Spindle_CAP/fmri_all_selected_N3.csv'

x_existed = 0

# pca_on = 1

# newfolder =  dir_root + '/Spindle_CAP/N20_informed_N11'
newfolder =  dir_root + '/Spindle_CAP/withinSBJ_CoRe_022'

if os.path.isdir(newfolder):
    print('< folder exists > : ' + newfolder)
else:
    os.mkdir(newfolder)
            
#=================== [ loop through subjects, concatenating the spindle-related TRs first ] ====================#

if 'fmri_all_selected' in locals():
    del fmri_all_selected

if x_existed == 0 and load_fmri_mat !=1: # initiate the spindle-TR extraction step.
    
    for sbj in sbjlist:

        print(' ')
        print(' ')
        print('  >>  Loading data on subject:  ' + sbj )
        print(' ')
        print(' ')

        if 'fmri_masked' in locals():
            del fmri_masked
        if 'fmri_1sess_selected' in locals():
            del fmri_1sess_selected
        if 'fmri_1sbj_selected' in locals():
            del fmri_1sbj_selected        



        # select and store the fmri data file names
        curr_sleep_fmri = []
        for day in daylist:
            for dr in os.listdir(dir_root + '/MRI_PreP/' + sbj + '/'+ day):
                if dr[-5:] == 'sleep': #
                    curr_sleep_fmri.append(day + '/' + dr)

        # select and store the spindle mark file names
        curr_spinlde_mark = []
        for dr in os.listdir(dir_root + '/Spindles/CoRe_spindles/output/'):
            if dr[0:8] == sbj and dr[-4:] == '.txt': #
                curr_spinlde_mark.append(dr)       


        # if numbers not matched, go check and delete based on the special sbj reference list.   
        if len(curr_spinlde_mark) < len(curr_sleep_fmri):
            print(curr_sleep_fmri)
            print('  - oops, ' + sbj + ' has an inconsistent match of numbers of fmri data and spindle mark files')

            if np.sum([ref_special['sbj_ID'] == sbj]) > 0: # means at least one row contains the info for this sbj
                sess_selected = ref_special['fMRI_sess_2use'].values[ref_special['sbj_ID'] == sbj][0]
                curr_sleep_fmri = [curr_sleep_fmri[sess_selected]]
                print('  - personalized file selection : [ ' + curr_sleep_fmri[0] + ' ]')
            else:
                print('  - oops, ' + sbj + ' does not have special reference information recorded!')

            if save_data_SZ == 1:                
                df_samples.specified_sleep_sess.iloc[sbjlist.index(sbj)] = sess_selected


        # load, filter, then concatenate fmri data    
        if len(curr_spinlde_mark) == len(curr_sleep_fmri):

            spindle_count = 0
            for ss in range(len(curr_sleep_fmri)):

                # load fmri data,
                train_path = gb.glob(dir_root + '/MRI_PreP/' + sbj + '/' + curr_sleep_fmri[ss])
                train_data = gb.glob(train_path[0] + '/*' + data_suffix) 
                print(train_data)

                # Z-scoring is done in this step as "standardize=True",
                spatial_mask = NiftiMasker(mask_img=mask_filename, standardize=True)
                fmri_masked = spatial_mask.fit_transform(train_data[0])

                # load spindle marks,
                spindle_mark = pd.read_csv(gb.glob(dir_root + '/Spindles/CoRe_spindles/output/' + curr_spinlde_mark[ss])[0], delimiter=' ')

                # filter the fmri by spindle marks; "spindle_mark"-1 since here 0=1,
                print(spindle_mark.shape)
                print(fmri_masked.shape[0])
# here
# not = before  spindle_mark = spindle_mark[spindle_mark.V_center < fmri_masked.shape[0]] # in case some marks are outside of fmri length
                spindle_mark = spindle_mark[spindle_mark.V_center <= fmri_masked.shape[0]] # in case some marks are outside of fmri length
                print(spindle_mark.shape)

                spindle_mark = spindle_mark-1
                spindle_count = spindle_count + spindle_mark.shape[0]

                fmri_1sess_selected = fmri_masked[np.unique(spindle_mark.V_center)]
# here

                ttt= np.unique(spindle_mark.V_center)
                print(ttt.shape)
                print(*ttt)


                # concatenate the selected fmri data within the sbj, eg core_022
                if ss == 0:                             # means this is the 1st session inside this loop.
                    fmri_1sbj_selected = fmri_1sess_selected
                else:                                   # not the first, so combine other sessions into one.
                    fmri_1sbj_selected = np.concatenate((fmri_1sbj_selected, fmri_1sess_selected), axis=0)


            if save_data_SZ == 1:    
                df_samples.total_spindle.iloc[sbjlist.index(sbj)] = spindle_count
                df_samples.total_TR.iloc[sbjlist.index(sbj)] = fmri_1sbj_selected.shape[0]
                df_samples.total_sleep_sess.iloc[sbjlist.index(sbj)] = len(curr_sleep_fmri)
        else:
            print('  - It is weird, ' + sbj + ' still has an inconsistent match')


        # concatenate the selected fmri data across the sbj
        if sbjlist.index(sbj) == 0:                    # means this is the 1st sbj inside this loop.
            fmri_all_selected = fmri_1sbj_selected
        else:                                          # not the first, so combine other sbjs into one.
            fmri_all_selected = np.concatenate((fmri_all_selected, fmri_1sbj_selected), axis=0)

    if save_fmri_mat == 1:  
        print('  - I am trying my best to save out the big matrix, be patient mate...')
        np.savetxt(fmri_mat_name, fmri_all_selected, delimiter=',')
    
    # save out sample size info   
    if save_data_SZ == 1:                
        name2give = newfolder + '/SpindleTR_SampleSize_N' + str(len(sbjlist)) + '.csv'
        if os.path.isfile(name2give):
            os.remove(name2give)
        df_samples.to_csv(name2give, sep='\t')  
    
if load_fmri_mat == 1: # just used existed matrix that already underwent the TR extraction steps.
    print('  - Loading the big matrix, no hurries...')
    fmri_all_selected = np.loadtxt(fmri_mat_name, delimiter=',')
    print(' ')
    print(80 * '+')



    
    
        
#=================== [ do the clustering ] ====================#




if x_existed == 0: # no existed or need to be updated
    X = fmri_all_selected
    del fmri_all_selected

for pca_on in range(0, 2):

    # <PCA, dimensionality reduction>
    if pca_on == 1:

#         X = full_X
        full_X = X

        pca = PCA(n_components=50)
#         pca = PCA(n_components=30)

        X = pca.fit_transform(X)
        # reduced_X.shape

        newfolder =  newfolder + '_pca'
        if os.path.isdir(newfolder):
            print('< folder exists > : ' + newfolder)
        else:
            os.mkdir(newfolder)

    # <pca help in finding proper seed to initiate> 
    # in this case the seeding of the centers is deterministic, hence we run the
    # kmeans algorithm only once with n_init=1

    # pca = PCA(n_components=n_digits).fit(data) # could get the center then initiate the seeding for clustering
    # bench_k_means(KMeans(init=pca.components_, n_clusters=n_digits, n_init=1),
    #               name="PCA-based",
    #               data=data)

#     range_n_clusters = range(2, len(sbjlist)+1)
    range_n_clusters = range(2, 10)

    df_scores = pd.DataFrame(np.zeros((len(range_n_clusters), 3)),
                       columns=['silhouette_avg', 'calinski_harabasz_score', 'davies_bouldin_score'])
    df_scores['clusters']=range_n_clusters


    for n_clusters in range_n_clusters:

        print('  ')
        print('  - solving for '+ str(n_clusters) + ' cluster plan...')

        del clusterer, cluster_labels, silhouette_avg, ch_score, db_score  

        # Create a subplot with 1 row and 2 columns
        fig, (ax1) = plt.subplots(1, 1)
        fig.set_size_inches(8, 7)

        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1 but in this example all
        # lie within [-0.1, 1]
        ax1.set_xlim([-0.1, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

        # Initialize the clusterer with n_clusters value and a random generator
        # seed of 10 for reproducibility.
        clusterer = KMeans(init='k-means++', n_clusters=n_clusters, random_state=10).fit(X)
    #     cluster_labels = clusterer.fit_predict(X)

    #     clusterer = KMeans(n_clusters=3, random_state=1).fit(X)
        cluster_labels = clusterer.labels_

        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters

        print(40 * '-')
        silhouette_avg = silhouette_score(X, cluster_labels)
        print("For n_clusters =", n_clusters,
              ", The average silhouette_score is :", silhouette_avg)

        ch_score = calinski_harabasz_score(X, cluster_labels)
        print("For n_clusters =", n_clusters,
              ", The calinski_harabasz_score is :", ch_score)

        db_score = davies_bouldin_score(X, cluster_labels)
        print("For n_clusters =", n_clusters,
              ", The davies_bouldin_score is :", db_score)

        df_scores.silhouette_avg.iloc[range_n_clusters.index(n_clusters)] = silhouette_avg
        df_scores.calinski_harabasz_score.iloc[range_n_clusters.index(n_clusters)] = ch_score
        df_scores.davies_bouldin_score.iloc[range_n_clusters.index(n_clusters)] = db_score

        print('  ')

        # Compute the silhouette scores for each sample
        sample_silhouette_values = silhouette_samples(X, cluster_labels)

        y_lower = 10
        for i in range(n_clusters):
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = \
                sample_silhouette_values[cluster_labels == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_clusters)
            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)

            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title("The silhouette plot for the various clusters")
        ax1.set_xlabel("The silhouette coefficient values")
        ax1.set_ylabel("Cluster label")

        # The vertical line for average silhouette score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])



    #     # 2nd Plot showing the actual clusters formed
    #     colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    #     ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
    #                 c=colors, edgecolor='k')

    #     # Labeling the clusters
    #     centers = clusterer.cluster_centers_
    #     # Draw white circles at cluster centers
    #     ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
    #                 c="white", alpha=1, s=200, edgecolor='k')

    #     for i, c in enumerate(centers):
    #         ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
    #                     s=50, edgecolor='k')

    #     ax2.set_title("The visualization of the clustered data.")
    #     ax2.set_xlabel("Feature space for the 1st feature")
    #     ax2.set_ylabel("Feature space for the 2nd feature")

        plt.suptitle(("Silhouette analysis for KMeans clustering, n_clusters = %d" % n_clusters),
                     fontsize=14, fontweight='bold')

        plt.savefig(newfolder + '/silhouette_plot_cluster=%d.png' % n_clusters)
        np.savetxt(newfolder + '/labels_cluster=%d.csv' % n_clusters, cluster_labels, delimiter=',')

        for cc in range(clusterer.cluster_centers_.shape[0]):
            curr_map = clusterer.cluster_centers_[cc, :]

            if pca_on == 1:
                curr_map = pca.inverse_transform(curr_map)


            cc_img = spatial_mask.inverse_transform(curr_map)
            cc_img.to_filename(newfolder + '/cluster=%d_cc%d_map.nii.gz' % (n_clusters, cc))

    plt.show()

    name2give = newfolder + '/evaluation_scores_N' + str(len(sbjlist)) + '.csv'
    if os.path.isfile(name2give):
        os.remove(name2give)
    df_scores.to_csv(name2give, sep='\t')  





< folder exists > : /Volumes/JDlab_Shuo/CoRe/Spindle_CAP/withinSBJ_CoRe_022
 
 
  >>  Loading data on subject:  CoRe_100
 
 
['/Volumes/JDlab_Shuo/CoRe/MRI_PreP/CoRe_100/D1/11-sleep/CoRe_100_D1_sleep_MDsCoNr%mniNLS.nii.gz']
(547, 3)
3369
(546, 3)
(458,)
72 115 123 124 143 154 174 183 197 198 199 200 202 204 205 209 219 229 234 240 241 242 253 254 257 259 261 264 275 276 284 288 290 291 294 300 301 303 305 313 316 321 322 324 325 327 330 331 341 349 354 355 372 376 377 378 380 384 389 392 395 397 398 399 412 416 417 418 420 427 428 429 431 433 434 435 437 438 439 440 449 451 455 463 465 467 468 469 476 477 494 498 505 506 507 508 509 519 531 541 542 556 558 569 580 584 585 587 588 589 590 593 595 605 606 609 618 619 620 621 622 623 647 648 650 656 667 669 672 675 701 702 708 723 725 733 735 736 741 742 743 745 746 747 752 765 766 767 781 783 787 797 799 802 810 815 827 857 861 867 871 873 876 880 883 888 897 903 911 925 932 940 973 988 999 1015 1034 1037 1061 1070 1106 1122 1130 1140 11

NameError: name 'clusterer' is not defined

In [220]:
len(curr_spinlde_mark)

4

In [221]:
curr_spinlde_mark

['CoRe_022_Day1_Night_01_spindles_TRnum_Pz_NREM23.txt',
 'CoRe_022_Day1_Night_02_spindles_TRnum_Pz_NREM23.txt',
 'CoRe_022_Day1_Night_01_spindles_TRnum_EEGmat_Pz_NREM23.txt',
 'CoRe_022_Day1_Night_02_spindles_TRnum_EEGmat_Pz_NREM23.txt']

In [68]:

sbjlist = [ "CoRe_001", "CoRe_023",  "CoRe_079", "CoRe_107",
"CoRe_082", "CoRe_100", "CoRe_087", "CoRe_192", "CoRe_195",
"CoRe_235", "CoRe_267", "CoRe_237", "CoRe_296",
"CoRe_011", "CoRe_022", "CoRe_050", "CoRe_054", "CoRe_162", "CoRe_155", "CoRe_094"] 
df_samples = pd.DataFrame(np.zeros((len(sbjlist), 3)),
                   columns=['total_TR', 'total_sleep_sess', 'specified_sleep_sess'])
df_samples['sbjlist']=sbjlist
df_samples.specified_sleep_sess+=-1

In [158]:
fmri_all_selected.shape

(369, 69881)

In [162]:
dataload-fmri_all_selected

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])